In [147]:
import pandas as pd
import numpy as np

from pulp import *

In [174]:
data_file = "data/fl_200_1"
data = pd.read_csv(data_file, sep=" ", names=["a", "b", "c", "d"], dtype={"a":float, "b":float, "c":float, "d":float})

In [175]:
data.head()

,a,b,c,d
0,200.0,200.0,NaN,NaN
1,226.0,600.0,34.824276,34.824276
2,312.0,600.0,34.905834,38.077609
3,379.0,600.0,43.586961,28.219700
4,599.0,600.0,40.395677,33.445805


In [176]:
n = int(data["a"][0])
m = int(data["b"][0])

In [177]:
facilities = data[1:(n+1)].rename({"a":"cost", "b":"capacity", "c":"x", "d":"y"}, axis="columns").reset_index(drop=True)
facilities.head()

,cost,capacity,x,y
0,226.0,600.0,34.824276,34.824276
1,312.0,600.0,34.905834,38.077609
2,379.0,600.0,43.586961,28.219700
3,599.0,600.0,40.395677,33.445805
4,280.0,600.0,35.072047,35.180733


In [178]:
customers = data[(n+1):].drop("d", axis=1).rename({"a":"demand", "b":"x", "c":"y"}, axis="columns").reset_index(drop=True)
customers.head()

,demand,x,y
0,2.0,47.668869,39.346976
1,1.0,42.675137,33.935762
2,5.0,7.495883,30.221757
3,5.0,49.947867,60.900421
4,4.0,55.565605,52.475980


We want to minimise the total cost of satifysing customer demand from a series of facilities (warehouses)

#### Description of rules and constraints

- Each customer must be served by exactly one facility
- The total demand of the customers cannot exceed the capacity of the facility

#### Notation and parameters

- $F$ is the set of facilities
- $C$ is the set of customers
- $f_i , \forall i\in F$ is the capacity of each facility
- $c_j , \forall j\in C$ is the demand of each customer
- $d_{ij} , \forall i \in F, j \in C$ is the euclidian distance between facility $j$ and customer $i$
- $s_i , \forall i\in F$ is the fixed cost for operating facility $i$

Note: In this example the euclidian distance between facility $j$ and customer $i$ is considered the transport cost so we can just add them to the fixed costs

#### Decision variables

$ x_{ij} \forall i\in F, \forall j\in C $ is whether customer $j$ is served by facility $i$

$x_{ij} \in \{0,1\}$

$p_i \forall i\in F$ is whether or not facility $i$ is used

$p_{i} \in \{0,1\}$

#### Objective function (minimize)

$$ \min (\sum_{i\in F}\sum_{j\in C}d_{ij}x_{ij} + \sum_{i \in F}p_i s_i) $$

#### Constraints

$$ \sum_{i \in F} x_{ij} \ge 1, \forall j \in C\; (1)$$

$$ \sum_{j \in C} x_{ij}c_{j} \le f_i, \forall i \in F\; (2)$$

$$ \sum_{j \in C} x_{ij} \le \ bigM p_i, \forall i \in F\; (3)$$

In [179]:
def euclidean_distance(x_1, x_2, y_1, y_2):
    return np.sqrt( np.square(x_1 - x_2)  + np.square(y_1 - y_2) )

F = list(range(0, n))
C = list(range(0, m))
f = np.array(facilities.capacity)
c = np.array(customers.demand)
s = np.array(facilities.cost)
d = np.zeros([n, m])

all_pairs = [(i, j) for i in F for j in C]

for pair in all_pairs:
    i = pair[0]
    j = pair[1]
    facility = facilities.iloc[[i]]
    customer = customers.iloc[[j]]
    d[i,j] = euclidean_distance(float(facility.x), float(customer.x), float(facility.y), float(customer.y))

In [180]:
### Create Problem/ Solver
prob = LpProblem("The facility location problem", LpMinimize)

### Create decision variables
xij = {}
for i in F:
    for j in C:
        xij[i,j] = LpVariable("x_%s_%s" % (i,j), 0, 1, LpBinary)
        
pi = {}
for i in F:
    pi[i] = LpVariable("p_%s" % (i), 0, 1, LpBinary)
    
## Set the objective function 
prob += lpSum([d[(i,j)]*xij[i,j] for i in F for j in C]) + lpSum([s[i] * pi[i] for i in F])

In [181]:
## Set constraint 1
for j in C:
    prob += lpSum([xij[i,j] for i in F]) >= 1,"%s must be assigned to at least one facility" % (j)

In [182]:
## Set constraint 2
for i in F:
    prob += lpSum([xij[i,j] * c[j] for j in C]) <= f[i],"%s demand must not exceed capacity" % (i)

In [183]:
## Set constraint 3
bigM=m
for i in F:
    prob += lpSum([xij[i,j] for j in C]) <= bigM * pi[i],"%s bigM constraint" % (i)

In [184]:
## Check model
#prob

In [185]:
import time

start = time.time()
## Solve model
prob.solve()
end = time.time()
print(pulp.LpStatus[prob.status], "solution is: ", float(end - start), "sec")

Optimal solution is:  110.43528318405151 sec


In [186]:
## Print variables with value greater than 0 
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

# Print The optimal objective function value
print("Total Cost = ", pulp.value(prob.objective))

p_68 = 1.0
x_68_0 = 1.0
x_68_1 = 1.0
x_68_10 = 1.0
x_68_100 = 1.0
x_68_101 = 1.0
x_68_102 = 1.0
x_68_103 = 1.0
x_68_104 = 1.0
x_68_105 = 1.0
x_68_106 = 1.0
x_68_107 = 1.0
x_68_108 = 1.0
x_68_109 = 1.0
x_68_11 = 1.0
x_68_110 = 1.0
x_68_111 = 1.0
x_68_112 = 1.0
x_68_113 = 1.0
x_68_114 = 1.0
x_68_115 = 1.0
x_68_116 = 1.0
x_68_117 = 1.0
x_68_118 = 1.0
x_68_119 = 1.0
x_68_12 = 1.0
x_68_120 = 1.0
x_68_121 = 1.0
x_68_122 = 1.0
x_68_123 = 1.0
x_68_124 = 1.0
x_68_125 = 1.0
x_68_126 = 1.0
x_68_127 = 1.0
x_68_128 = 1.0
x_68_129 = 1.0
x_68_13 = 1.0
x_68_130 = 1.0
x_68_131 = 1.0
x_68_132 = 1.0
x_68_133 = 1.0
x_68_134 = 1.0
x_68_135 = 1.0
x_68_136 = 1.0
x_68_137 = 1.0
x_68_138 = 1.0
x_68_139 = 1.0
x_68_14 = 1.0
x_68_140 = 1.0
x_68_141 = 1.0
x_68_142 = 1.0
x_68_143 = 1.0
x_68_144 = 1.0
x_68_145 = 1.0
x_68_146 = 1.0
x_68_147 = 1.0
x_68_148 = 1.0
x_68_149 = 1.0
x_68_15 = 1.0
x_68_150 = 1.0
x_68_151 = 1.0
x_68_152 = 1.0
x_68_153 = 1.0
x_68_154 = 1.0
x_68_155 = 1.0
x_68_156 = 1.0
x_68_157 = 1.0
x_68_158 

In [187]:
## Print variables with value greater than 0 
customer_assignments = {}
for v in prob.variables():
    if v.varValue > 0:
        var_split = v.name.split("_")
        if (var_split[0] == "x"):
            customer_assignments[var_split[2]] = var_split[1]

settings = [customer_assignments[str(i)] for i in range(0,m)]

In [188]:
# prepare the solution in the specified output format
output_data = str(pulp.value(prob.objective)) + ' ' + str(1) + '\n'
output_data += ' '.join(map(str, settings))
print(output_data)

3807.322699189734 1
68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68
